In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve

import joblib

In [17]:
df = pd.read_csv("cleaned_no_emojis.csv") 
X = df.drop("target_variable", axis=1)
y = df["target_variable"]
y = y.map({"Good Trader": 1, "Bad Trader": 0})

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

In [18]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [19]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", DecisionTreeClassifier(random_state=42))
])
pipeline

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [20]:

param_grid = {
    "classifier__criterion": ["gini", "entropy", "log_loss"],
    "classifier__max_depth": [3, 5, 7, 9, None],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 3, 5]
}

In [21]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)
grid_search

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'classifier__criterion': ['gini', 'entropy', ...], 'classifier__max_depth': [3, 5, ...], 'classifier__min_samples_leaf': [1, 3, ...], 'classifier__min_samples_split': [2, 5, ...]}"
,scoring,'roc_auc'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [22]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'classifier__criterion': ['gini', 'entropy', ...], 'classifier__max_depth': [3, 5, ...], 'classifier__min_samples_leaf': [1, 3, ...], 'classifier__min_samples_split': [2, 5, ...]}"
,scoring,'roc_auc'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [23]:
print(f"Best Params: {grid_search.best_params_}")
print(f"Best CV ROC-AUC: {grid_search.best_score_:.3f}")

Best Params: {'classifier__criterion': 'gini', 'classifier__max_depth': 3, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
Best CV ROC-AUC: 0.998


In [24]:
best_pipeline = grid_search.best_estimator_
best_pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [25]:
y_probs = best_pipeline.predict_proba(X_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
print(f"Best threshold (val): {best_threshold:.3f}, Max F1: {f1_scores[best_idx]:.3f}")

Best threshold (val): 1.000, Max F1: 1.000


In [26]:
best_pipeline.best_threshold_ = best_threshold

In [27]:
y_pred = (best_pipeline.predict_proba(X_test)[:, 1] >= best_threshold).astype(int)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1:", f1_score(y_test, y_pred))
print("Test ROC-AUC:", roc_auc_score(y_test, best_pipeline.predict_proba(X_test)[:, 1]))

Test Accuracy: 1.0
Test F1: 1.0
Test ROC-AUC: 1.0


In [32]:
# Check test set size
print("Test size:", len(X_test))

# Check class balance
print("Class balance:", np.bincount(y_test))

# Check correlation with target
# Ensure target column is present
temp_df = X.copy()
temp_df["target"] = y.values  # explicitly add it back

# Compute correlations safely
corrs = temp_df.corr(numeric_only=True)["target"].sort_values(ascending=False)
print("Top correlated features with target:\n", corrs.head(10))



Test size: 400
Class balance: [200 200]
Top correlated features with target:
 target           1.000000
active_weeks     0.542483
total_volume     0.057271
tx_count_365d    0.046488
Name: target, dtype: float64


In [33]:
joblib.dump(best_pipeline, "tree_pipeline_tuned.pkl")
print("Pipeline saved to tree_pipeline_tuned.pkl")

Pipeline saved to tree_pipeline_tuned.pkl
